In [3]:
import os
import torch
import argparse
from train.rm import *
from safetensors.torch import load_file
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import (
    SAFE_WEIGHTS_NAME,
)

In [6]:
def fix_valuehead(
    model, rm_ckpt_dir: str, rm_save_dir: str, V_HEAD_WEIGHTS_NAME: str = "value_head.bin"
) -> None:

    path_to_checkpoint = os.path.join(rm_ckpt_dir, SAFE_WEIGHTS_NAME)
    # state_dict = load_file(path_to_checkpoint)
    rm_model = AutoModelForCausalLM.from_pretrained(
        rm_ckpt_dir
    )
    state_dict = model.state_dict()

    
    tokenizer = AutoTokenizer.from_pretrained(
        rm_ckpt_dir, 
        trust_remote_code=True, 
        use_fast=True,
        padding_side="right",
        split_special_tokens=False,
    )
    model.pretrained_model.resize_token_embeddings(len(tokenizer))
    decoder_state_dict = {}
    v_head_state_dict = {}
    for name, param in state_dict.items():
        if name.startswith("v_head."):
            v_head_state_dict[name] = param
        else:
            decoder_state_dict[name.replace("pretrained_model.", "", 1)] = param

    model.pretrained_model.save_pretrained(
        rm_save_dir, state_dict=decoder_state_dict or None
    )
    tokenizer.save_pretrained(rm_save_dir)
    torch.save(v_head_state_dict, os.path.join(rm_save_dir, V_HEAD_WEIGHTS_NAME))
    #os.remove(path_to_checkpoint)

In [7]:
base_dir = '/groups/kjun/tnn/datasets/'
    
# dataset path
data_dir = base_dir + "/prm800k/math_splits"

llm_tokenizer_dir = base_dir + "/Llama-3.2-1B-Instruct"
prm_tokenizer_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data"
# prm_tokenizer_dir = base_dir + "/Llama-3.2-1B-RM-GSM8k"



args = argparse.Namespace()
args.custom_cfg = f"config/sft_eval_mcts.yaml"
args.qaf = f"eval_data/math500_test.json"
args.sft_model_path = f"{llm_tokenizer_dir}"
args.rm_ckpt_path = f"{prm_tokenizer_dir}"
args.rm_save_path = f"results/Llama3.1-8B-PRM-Deepseek-Data"

model = AutoModelForCausalLM.from_pretrained(
    args.sft_model_path, 
    trust_remote_code=True,
    #torch_dtype=torch.bfloat16,
    use_cache=False,
)

model = RewardModelWithValueHead(pretrained_model=model)
fix_valuehead(model, args.rm_ckpt_path, args.rm_save_path)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]